This notebook will accumulate the final global stricture of iterating through County GDBs. 

In [ ]:
import glob
select_gdb = '48113'
for gdb in glob.glob('D:\Projects\WaterSurfacePolygons\Data\CountyEdge_Testing\*.gdb'):
    if os.path.basename(gdb).split('.')[0] == select_gdb:
        selFZ = gpd.read_file(gdb, driver='FileGDB', layer='FLOOD_ZONES')
        selBFE = gpd.read_file(gdb, driver='FileGDB', layer='DYNAMIC')

        sel_ae = selFZ.loc[selFZ['D_ZONE'] == 'AE']
        sel_ae_dynamic = sel_ae.loc[sel_ae['BFE_STATIC'].isnull()]
        sel_ae_dynamic['FIPS'] = sel_ae_dynamic.apply(lambda x: str(x['STATE_CODE']) + str(x['COUNTY_CODE']), axis=1)

        sel_ae_dis = sel_ae_dynamic.dissolve(by='FIPS')
        sel_ae_dis.to_crs(26913, inplace=True)
        sel_fsp = sel_ae_dis[['geometry']]
        sel_fsp.reset_index(inplace=True)

        # Reproject BFEs 
        selBFE.to_crs(26913, inplace=True)
        sel_bfe = selBFE[['ELEV', 'geometry']]
        """ Main List of adjcaent Co Fips """
        #adj_ls = adj.loc[adj['FIPSSTCO'] == select_gdb, 'n_fips'].to_list()
        #adj_ls = [fips for adj in adj_ls for fips in adj]

        """ Test List of adjacent Co FIPS """
        adj_ls = ['48439'] # Only tarrant
        for a in adj_ls:
            for adj_gdb in glob.glob('D:\Projects\WaterSurfacePolygons\Data\CountyEdge_Testing\*.gdb'):
                if os.path.basename(adj_gdb).split('.')[0] == a:
                    adjFZ = gpd.read_file(adj_gdb, driver='FileGDB', layer='FLOOD_ZONES')
                    adjBFE = gpd.read_file(adj_gdb, driver='FileGDB', layer='DYNAMIC')

                    adj_ae = adjFZ.loc[adjFZ['D_ZONE'] == 'AE']
                    adj_ae_dynamic = adj_ae.loc[adj_ae['BFE_STATIC'].isnull()]
                    adj_ae_dynamic['FIPS'] = adj_ae_dynamic.apply(lambda x: str(x['STATE_CODE']) + str(x['COUNTY_CODE']), axis=1)

                    adj_ae_dis = adj_ae_dynamic.dissolve(by='FIPS')
                    adj_ae_dis.to_crs(26913, inplace=True)
                    adj_fsp = adj_ae_dis[['geometry']]
                    adj_fsp.reset_index(inplace=True)

                    # Reproject BFEs 
                    adjBFE.to_crs(26913, inplace=True)
                    adj_bfe = adjBFE[['ELEV', 'geometry']]

        print('Merge All County Flood Zones')
        all_fsp = pd.concat([sel_fsp, adj_fsp], ignore_index=True)
        all_fsp_x = all_fsp.explode()
        all_fsp_merge = g(MultiPolygon(all_fsp_x.geometry.to_list()), crs=26913)

        print('Merge All County BFEs')
        all_bfe = pd.concat([sel_bfe, adj_bfe], ignore_index=True)

        print('Clean BFEs')
        bfe_bfe = all_bfe.sjoin(all_bfe, how='left')
        bfe_bfe.reset_index(inplace=True)
        brok = bfe_bfe.loc[bfe_bfe['index'] != bfe_bfe['index_right']]
        if brok.shape[0] > 0:
            print('Cleaning BFEs...')
            brok_geoms = brok[['index', 'geometry']]
            fix = brok.merge(brok_geoms, left_on='index_right', right_on='index')
        
            fix['new_geom'] = fix.apply(lambda x: linemerge([m for m in x.geometry_x] + [m for m in x.geometry_y]), axis=1)
            fix_gpd = gpd.GeoDataFrame(fix[['index_x', 'ELEV_left', 'new_geom']], geometry='new_geom', crs=26913)
            fix_gpd.rename(columns={'index_x': 'index', 'ELEV_left': 'ELEV', 'new_geom': 'geometry'}, inplace=True)
            fix_gpd.set_index('index', inplace=True)
            bfe_clean = all_bfe.loc[~all_bfe.index.isin(fix_gpd.index)]
            bfe = pd.concat([bfe_clean, fix_gpd])
            bfe = bfe.drop_duplicates('geometry')

        # Extend BFEs over FSP Poly then reset bfe to new position
        print('Extending BFE...')
        bfe_ext = bfe_extend(bfe, all_fsp_merge)

        print('Clipping BFE...')
        bfe = bfe_ext.overlay(all_fsp_merge)

        print('Splitting FSP...')
        # Split FSP Poly by extended BFEs
        fsp_s = split_fsp(all_fsp_merge, bfe_ext)

        print('Cleaning FSP...')
        # Remove slivers due to overlaps from extended BFEs
        fsp_s = fsp_s.overlay(all_fsp_merge)
        fsp_s = fsp_s.loc[fsp_s.geom_type == 'Polygon']
        fsp_s = fsp_s[['geometry']]
        fsp_s.reset_index(inplace=True)

            



